In [167]:
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import numpy as np
import re
import sys
import time
import random

In [168]:
class box_score_scraping:
    def __init__(self, driver, game_id, url, away_team, home_team):
        self.home_team = home_team
        self.away_team = away_team
        self.game_id = game_id
        self.player_dict = {}
        self.driver = driver

        
        self.url = url

    def get_box_score(self):
        self.driver.get(self.url)
        time.sleep(random.uniform(0.5, 3))
        col_names = []

        player_dict = {"Game_ID": self.game_id, "Player_Name": [], "Player_ID": [], "Starter": [], "Home": []}

        try:
            WebDriverWait(self.driver, 5, poll_frequency=0.1).until(
                EC.presence_of_element_located((By.ID, f"div_box-{self.home_team}-game-basic"))
                )
            # WebDriverWait(self.driver, 10, poll_frequency=0.1).until(
            #     EC.presence_of_element_located((By.ID, "content"))
            #     )
            home_table = self.driver.find_element(By.ID, f"div_box-{self.away_team}-game-basic")
            away_table = self.driver.find_element(By.ID, f"div_box-{self.away_team}-game-basic")


            # away_table = away_table = WebDriverWait(self.driver, 20).until(
            #     EC.presence_of_element_located((By.ID, f"div_box-{self.away_team}-game-basic"))
            # )
        except TimeoutException as e:
            raise e
        
        rows = home_table.find_elements(By.TAG_NAME, "tr")
        starter = 0
        for row in rows:
            starter += 1

            if (starter == 2): #These are the column names
                col_names = row.text.split()[1:] #Get rid of the starters column name because we're just using player name instead
                for i in col_names:
                    player_dict[i] = []
            
            jawn = row.find_elements(By.CSS_SELECTOR, "th")
            player_id = jawn[0].get_attribute("data-append-csv")
            player_name = jawn[0].text
            if player_id is None:
                continue #This is one of the descriptor or title rows, not the things we want
            data = row.find_elements(By.TAG_NAME, "td")
            if len(data) == 1: #DID NOT PLAY, WE DON'T COUNT THEM
                continue

            #We have the data, just need to add it to the dictionary properly
            player_dict["Player_ID"].append(player_id)
            player_dict["Home"].append(True)
            player_dict["Player_Name"].append(player_name)
            player_dict["Starter"].append(True if starter <= 7 else False)

            for i in range(len(col_names)):
                player_dict[col_names[i]].append(data[i].text)

        
        rows = away_table.find_elements(By.TAG_NAME, "tr")
        starter = 0
        for row in rows:
            starter += 1
            
            jawn = row.find_elements(By.CSS_SELECTOR, "th")
            player_id = jawn[0].get_attribute("data-append-csv")
            player_name = jawn[0].text
            if player_id is None:
                continue #This is one of the descriptor or title rows, not the things we want
            data = row.find_elements(By.TAG_NAME, "td")
            if len(data) == 1: #DID NOT PLAY, WE DON'T COUNT THEM
                continue

            #We have the data, just need to add it to the dictionary properly
            player_dict["Player_ID"].append(player_id)
            player_dict["Home"].append(False)
            player_dict["Player_Name"].append(player_name)
            player_dict["Starter"].append(True if starter <= 7 else False)

            for i in range(len(col_names)):
                player_dict[col_names[i]].append(data[i].text)



        self.player_dict = player_dict
    
    def append_data(self):
        df = pd.DataFrame.from_dict(self.player_dict)
        df.to_csv("../data/BoxScoreData.csv", mode = 'a', header=False)

In [86]:
df = pd.read_csv("../data/ScheduleData2.csv")

In [187]:
team_to_abbrev = {
    "New York Knicks": "NYK",
    "Dallas Mavericks": "DAL",
    "Los Angeles Lakers": "LAL",
    "Oklahoma City Thunder": "OKC",
    "Golden State Warriors": "GSW",
    "Cleveland Cavaliers": "CLE",
    "Washington Wizards": "WAS",
    "Orlando Magic": "ORL",
    "Indiana Pacers": "IND",
    "Charlotte Bobcats": "CHA",
    "Minnesota Timberwolves": "MIN",
    "San Antonio Spurs": "SAS",
    "Phoenix Suns": "PHO",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "New Jersey Nets": "NJN",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Toronto Raptors": "TOR",
    "Atlanta Hawks": "ATL",
    "Detroit Pistons": "DET",
    "New Orleans Hornets": "NOH",
    "Memphis Grizzlies": "MEM",
    "Denver Nuggets": "DEN",
    "Houston Rockets": "HOU",
    "Boston Celtics": "BOS",
    "Utah Jazz": "UTA",
    "Los Angeles Clippers": "LAC",
    "Chicago Bulls": "CHI",
    "Philadelphia 76ers": "PHI",
    "Brooklyn Nets": "BRK",
    "New Orleans Pelicans": "NOP",
    "Charlotte Hornets": "CHO"
} #Used to convert the team name to the abbreviation used by basketabll reference

In [ ]:
last_index_df = pd.read_csv("../data/BoxScoreData.csv")
final_index = max(last_index_df.iloc[-1].Game_ID, 10699) #Had to push the index back, taking all the games from 2012 to now was taking too long :(

df = df[pd.to_numeric(df["game_id"], errors="coerce").notna()] #get rid of the stupid rows that are just repeats of the column names
remaining_df = df.loc[df["game_id"].astype(int) >= int(final_index)] #It broke down, starting from where we left off. Will have to delete duplicate rows later 




options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--disable-extensions")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-notifications")
options.add_argument("--disable-infobars")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

#Now an option that actually has heads 
head_options = webdriver.ChromeOptions()
head_options.add_argument("--disable-blink-features=AutomationControlled")
head_options.add_argument("--disable-gpu")
head_options.add_argument("--no-sandbox")
head_options.add_argument("--disable-extensions")
head_options.add_argument("--disable-dev-shm-usage")
head_options.add_argument("--disable-popup-blocking")
head_options.add_argument("--disable-notifications")
head_options.add_argument("--disable-infobars")
head_options.add_experimental_option("excludeSwitches", ["enable-automation"])
head_options.add_experimental_option('useAutomationExtension', False)
head_options.add_experimental_option("prefs", prefs)


driver = webdriver.Chrome(options=head_options)

games = 0

error = 0
success = 0
for index, row in remaining_df.iterrows():
    if np.abs(int(row["Away Score"]) - int(row["Home Score"])) > 10: #Only take close games from now on...
        continue
    bss = box_score_scraping(driver, row["game_id"], row["url"], team_to_abbrev[row["Away Team"]], team_to_abbrev[row["Home Team"]])
    try:
        bss.get_box_score()
            
        bss.append_data()
        error = 0
        games += 1
        success += 1
        print(games)
        #We're just gonna do it with the head from now on, makes it work so much better even if it's slightly slower
        # if (success == 8): #After 8 consecutive successes, stop printing the head
        #     driver.quit()
        #     driver = webdriver.Chrome(options=options)

        if (games % 40 == 0): #restart the driver every so often, it seems to be slowing down heavily
            driver.quit()
            driver = webdriver.Chrome(options=head_options)
            success = 0


    except Exception as e:
        success = 0
        error += 1
        print(e)
        print(row["url"])

        if (error > 5): # We got nothing returned 5 times in a row, let it rest
            print("chilling")
            time.sleep(10)
            error = 0
            driver.quit()
            driver = webdriver.Chrome(options=head_options)
        continue
driver.quit()

In [197]:
#We messed up, got the abbreviation wrong for the hornets and the nets, so with the updated abbreviations we'll add those games as well.
missing_id_df = pd.read_csv("../data/BoxScoreData.csv")
current_ids = list(set(missing_id_df["Game_ID"]))
potential_ids = [str(i) for i in range(10699, max(current_ids)) if i not in current_ids] #These are the games we missed the first time


In [198]:
driver = webdriver.Chrome(options=head_options)

games = 0

error = 0
success = 0
missing_ids = df.loc[df.game_id.isin(potential_ids)]
for index, row in missing_ids.iterrows():
    if np.abs(int(row["Away Score"]) - int(row["Home Score"])) > 10: #Only take close games from now on...
        continue
    bss = box_score_scraping(driver, row["game_id"], row["url"], team_to_abbrev[row["Away Team"]], team_to_abbrev[row["Home Team"]])
    try:
        bss.get_box_score()
            
        bss.append_data()
        error = 0
        games += 1
        success += 1
        print(games)
        #We're just gonna do it with the head from now on, makes it work so much better even if it's slightly slower
        # if (success == 8): #After 8 consecutive successes, stop printing the head
        #     driver.quit()
        #     driver = webdriver.Chrome(options=options)

        if (games % 40 == 0): #restart the driver every so often, it seems to be slowing down heavily
            driver.quit()
            driver = webdriver.Chrome(options=head_options)
            success = 0


    except Exception as e:
        success = 0
        error += 1
        print(e)
        print(row["url"])

        if (error > 5): # We got nothing returned 5 times in a row, let it rest
            print("chilling")
            time.sleep(10)
            error = 0
            driver.quit()
            driver = webdriver.Chrome(options=head_options)
        continue
driver.quit()

Python(54950) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(54979) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


Python(55074) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Python(55103) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
